[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/reznikmm/ada-howto/advent-2020?filepath=%2Fhome%2Fjovyan%2Fnb%2F07%2F07.ipynb)

# Day 7

Firstly, let's force assertion checking. We will use `pragma Assert` in the code.

In [1]:
pragma Assertion_Policy (Check);

We need some standard packages to read the text file and build an index of bags.Let's force assertion checking:

In [2]:
with Ada.Text_IO;
with Ada.Strings.Unbounded;
with Ada.Strings.Maps;
with Ada.Strings.Unbounded.Hash;
with Ada.Containers.Hashed_Maps;
with Ada.Containers.Vectors;

To make reading rules simplier let's introduce a Tokenizer. It returns input word by word. Each word is separated by spaces. It returns an empty string at the end of file.

In [3]:
package Tokenizer is
   procedure Open_File (Name : String);
   function Next_Word return String;  --  Get "" on end of file
private
   Input  : Ada.Text_IO.File_Type;
   Buffer : Ada.Strings.Unbounded.Unbounded_String;
   From   : Positive := 1;
   Space  : constant Ada.Strings.Maps.Character_Set :=
     Ada.Strings.Maps.To_Set (" ");
end Tokenizer;

package body Tokenizer is
      
   function Next_Word return String is
      First : Positive;
      Last  : Natural;
   begin
      loop
         Ada.Strings.Unbounded.Find_Token
           (Source => Buffer,
            Set    => Space,
            From   => From,
            Test   => Ada.Strings.Outside,
            First  => First,
            Last   => Last);

         if First <= Last then
            From := Last + 1;
            return Ada.Strings.Unbounded.Slice (Buffer, First, Last);
         elsif Ada.Text_IO.End_Of_File (Input) then
            return "";
         else
            Buffer := Ada.Strings.Unbounded.To_Unbounded_String
              (Ada.Text_IO.Get_Line (Input));
            From := 1;
         end if;
      end loop;
   end Next_Word;

   procedure Open_File (Name : String) is
   begin
      Ada.Text_IO.Open (Input, Ada.Text_IO.In_File, Name);
   end Open_File;
end Tokenizer;

Now let's define a type for bag color. I cheat a bit by using a subtype. It simplifies program a bit.

In [4]:
subtype Color is Ada.Strings.Unbounded.Unbounded_String;

function To_Color (Text : String) return Color
  renames Ada.Strings.Unbounded.To_Unbounded_String;

A color and count describes a bag nested in some other bag.

In [5]:
type Nested is record
   Kind  : Color;
   Count : Positive;
end record;

To keep several such descritions we naad a vector.

In [6]:
package Nested_Vectors is new Ada.Containers.Vectors (Positive, Nested);

And now let's create a map color to descrition.

In [7]:
use type Color;

package Color_Maps is new Ada.Containers.Hashed_Maps
  (Key_Type        => Color,
   Element_Type    => Nested_Vectors.Vector,
   Hash            => Ada.Strings.Unbounded.Hash,
   Equivalent_Keys => "=",
   "="             => Nested_Vectors."=");

Map : Color_Maps.Map;

With a simple procedure we can read a color by reading two words.

In [8]:
procedure Read_Color (Value : out Color) is
begin
   Value := Ada.Strings.Unbounded.Null_Unbounded_String;
   Ada.Strings.Unbounded.Append (Value, Tokenizer.Next_Word);
   Ada.Strings.Unbounded.Append (Value, " ");
   Ada.Strings.Unbounded.Append (Value, Tokenizer.Next_Word);
end Read_Color;

Another procedure reads bag count by reading a word. It has an exception to be able to read 'no' as zero.

In [9]:
procedure Read_Count (Count : out Natural) is
   Word : String := Tokenizer.Next_Word;
begin
   if Word = "no" then
      Count := 0;
   else
      Count := Positive'Value (Word);
   end if;
end Read_Count;

Now we are ready to read rules into the map.

In [10]:
Tokenizer.Open_File ("/home/jovyan/md/07/input");
   
loop
   declare
      use type Color;
      Vector : Nested_Vectors.Vector;
      Outer  : Color;
   begin
      Read_Color (Outer);
      exit when Outer = To_Color (" ");

      pragma Assert (Tokenizer.Next_Word = "bags");
      pragma Assert (Tokenizer.Next_Word = "contain");

      declare
         Inner : Color;
         Count : Natural;
      begin
         Read_Count (Count);
         if Count = 0 then
            pragma Assert (Tokenizer.Next_Word = "other");
            pragma Assert (Tokenizer.Next_Word = "bags.");
         else
            loop
               Read_Color (Inner);
               Vector.Append ((Inner, Count));

               declare
                  Bag : String := Tokenizer.Next_Word;
               begin
                  pragma Assert (Bag (Bag'Last) in '.' | ',');
                  exit when Bag (Bag'Last) = '.';
                  Read_Count (Count);
                end;
            end loop;
         end if;

         Map.Insert (Outer, Vector);
      end;
   end;
end loop;

We will count result using a function, that count a bag and all nested bags.

In [11]:
function Count_Bags (Map : Color_Maps.Map; Value : Color) return Positive is
   Result : Positive := 1;
begin
   if Map.Contains (Value) then
      for Item of Map (Value) loop
         Result := Result + Item.Count * Count_Bags (Map, Item.Kind);
      end loop;
   end if;
      
   return Result;
end Count_Bags;

Now call it for our "shiny gold" bag.

In [12]:
Result : Positive := Count_Bags (Map, To_Color ("shiny gold"));

Decrement `Result`, because we shouldn't count "shiny gold" bag itself.

In [13]:
Result := Result - 1;

Ada.Text_IO.Put_Line (Result'Image);

 45018


----
[Back to Table of Contents](https://github.com/reznikmm/ada-howto/tree/advent-2020)